In [ ]:
import tensorflow as tf

tf.debugging.set_log_device_placement(True)

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())


In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import Flatten,BatchNormalization

In [3]:
# 사전 학습된 모델 불러오기
input_tensor = Input(shape=(224,224,3))
vgg_model = VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)

vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
vgg_model.trainable=False
        
flatten_layer = Flatten()

dense_layer_1 = Dense(256, activation='relu')

do= layers.Dropout(0.2)

bm1=BatchNormalization()


prediction_layer  = Dense(149, activation='softmax')
        
model = Sequential([
    vgg_model,
    flatten_layer,
    dense_layer_1,
    do,
    bm1,
    prediction_layer
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               6422784   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 149)               38293     
Total params: 21,176,789
Trainable params: 6,461,589
Non-trainable params: 14,715,200
____________________________________

In [5]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy',
                     optimizer=RMSprop(1e-4),
                     metrics=['accuracy'])

In [9]:
train_dir ='D:/빅데이터기반지능형서비스개발/06ai서비스개발pjt/data_split/train'
test_dir = 'D:/빅데이터기반지능형서비스개발/06ai서비스개발pjt/data_split/val'

train_image_generator = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,
                                          horizontal_flip=True,
                                          width_shift_range=.2,height_shift_range=.2,
                                          fill_mode='nearest')

val_image_generator = ImageDataGenerator(rescale=1./255)


                                        
train_data_gen = train_image_generator.flow_from_directory(batch_size=32,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(224, 224),
                                                           class_mode='categorical')

val_data_gen = val_image_generator.flow_from_directory(batch_size=32,
                                                         directory=test_dir,
                                                         target_size=(224, 224),
                                                         class_mode='categorical')

Found 95475 images belonging to 149 classes.
Found 24049 images belonging to 149 classes.


In [10]:
# 모델 학습

history = model.fit_generator(train_data_gen,
                                epochs=6*10,
                              steps_per_epoch=500,
                                validation_data=val_data_gen,
                              validation_steps=100,
                              shuffle=True)


model.save("logs/aug_VGG16_try_6_epo_5.h5")


Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 500 steps, validate for 100 steps
Epoch 1/60
  1/500 [..............................] - ETA: 10:36:46

KeyboardInterrupt: 

In [ ]:
# 최종 결과 리포트
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

import matplotlib.pyplot as plt

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='testing acc')
plt.title('Training and testing accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='testing loss')
plt.title('Training and testing loss')
plt.legend()

plt.show()

In [ ]:
rt='D:\\빅데이터기반지능형서비스개발\\06ai서비스개발pjt\\data_split\\test\\'
classes=os.listdir(rt)


In [ ]:
from PIL import Image

for i in range(len(classes)):
    tmp_rt=rt+classes[i]
    d=os.listdir(tmp_rt)[0]   
    print(tmp_rt+'\\'+d)

    tst = Image.open(tmp_rt+'\\'+d)
    tst=tst.resize((224,224))
    tst=np.expand_dims(tst,axis=0)
    prd=model.predict(tst)
    if np.argmax(prd[0])==i:
        print('prd',np.argmax(prd[0]),classes[np.argmax(prd[0])],'real',i,classes[i])
    

In [ ]:
new_model=model
prd=new_model.predict(val_data_gen[0][0])[0]
np.argmax(prd[0])

In [ ]:
# 모델 불러오기

from tensorflow.keras.models import load_model

new_model = load_model("./logs/aug_VGG16_try_2_epo_10.h5",compile=False)
model.summary()